We will be implementing a CNN from scratch heere using just Numpy. Here, I will keep the entire evolution of code until we get a final, polished CNN architecture. This way, we can understand the problems and implementations in a chronological way. 

In [15]:
import numpy as np
import random
import itertools



In [5]:
#Define the fundamentals of CNN:

#Take W size Input Image.
#Use a filter of size K.
#Return output of size W-k+1 (No Padding, No Stride).
#We will use and understand Padding & Stride as we progress into bottlenecks and more complex problems. 
#The main task here is to develop algorithm to convolute the filter through the image.

class con2d:
    def __init__(self, w, k):
        self.W = w
        self.K = k
        self.filter = np.random.rand(k,k)
        self.bias = random.uniform(-1,1)

    def forward(self, X):
        sum = 0
        Y = np.zeros((self.W + 1 - self.K, self.W + 1 - self.K))
        for i in range(self.W + 1 - self.K):
            for j in range(self.W + 1 - self.K):
                patch = X[i:i+self.K, j:j+self.K]
                for a in range(self.K):
                    for b in range(self.K):
                        sum += patch[a,b] * self.filter[a,b]
                #sum += self.bias
                #sum = np.tanh(sum)
                Y[i,j] = sum
                sum = 0
        return Y

In [6]:
input_image = np.array([
    [1, 1, 1, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 0],
    [0, 1, 1, 0, 0]
])

kernel = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

myCNN = con2d(5,3)
myCNN.filter = kernel
myCNN.forward(input_image)

array([[-2.,  0.,  2.],
       [-3., -2.,  2.],
       [-3., -1.,  2.]])

This was an extremely inefficient implementation but we did get correct outcome. For now lets focus on the fact that CNNs work on rgb data which has 3 channels. But our code assumes an image as a flat 2D surface. We have to add depth/channel to our input matrix and thus our kernel. But importantly, our output matrix/activation map must remain the same.

In [7]:
class con2d:
    def __init__(self, w, k, c):
        self.W = w
        self.K = k
        self.C = c #Channel
        self.filter = np.random.rand(k,k,c)
        self.bias = random.uniform(-1,1)

    def forward(self, X):
        sum = 0
        Y = np.zeros((self.W + 1 - self.K, self.W + 1 - self.K))
        for i in range(self.W + 1 - self.K):
            for j in range(self.W + 1 - self.K):
                patch = X[i:i+self.K, j:j+self.K, :]
                for a in range(self.K):
                    for b in range(self.K):
                        for c in range(self.C):
                            sum += patch[a,b,c] * self.filter[a,b,c]
                #sum += self.bias
                #sum = np.tanh(sum)
                Y[i,j] = sum
                sum = 0
        return Y

In [8]:
input_vol = np.zeros((4, 4, 2))
input_vol[:, :, 0] = 1
input_vol[:, :, 1] = 2

kernel = np.zeros((3, 3, 2))
kernel[:, :, 0] = 1
kernel[:, :, 1] = -1

myCNN = con2d(4,3,2)
myCNN.filter = kernel
myCNN.forward(input_vol)

array([[-9., -9.],
       [-9., -9.]])

In [9]:
#To make this complete, lets uncomment bias application and let's try to have more than one filters.
class con2d:
    def __init__(self, w, k, c, n):
        self.W = w
        self.K = k
        self.C = c #Channels
        self.Cn = n #Number of filters
        self.filters = [np.random.rand(k,k,c) for _ in range(n)]
        self.bias = [random.uniform(-1,1) for _ in range(n)]

    def forward(self, X):
        sums = np.zeros(self.Cn)
        Y = [np.zeros((self.W + 1 - self.K, self.W + 1 - self.K)) for _ in range(self.Cn)]
        for i in range(self.W + 1 - self.K):
            for j in range(self.W + 1 - self.K):
                patch = X[i:i+self.K, j:j+self.K, :]
                for a in range(self.K):
                    for b in range(self.K):
                        for c in range(self.C):
                            for index, filter in enumerate(self.filters):
                                sums[index] += patch[a,b,c] * filter[a,b,c]
                for idx, sum in enumerate(sums):
                    sum += self.bias[idx]
                    #sum = np.tanh(sum)
                    Y[idx][i,j] = sum
                    sum = 0
        return Y

But these nested loops are extremely inefficient. We have to adress this. Modern libraries like pytorch and tensorflow use a standard practice called "im2col" where the entire image is stretched into a single vector arranged by the filter size. One way is to then also dilate our filter adding zeros in between such that a single matrix multiplication will simulate the filter being slid over the image. 

This however, would be inefficient. Why? We aren't using multiple nested loops which significantly improve performance. But, we are doing unecessarily large matrix multiplication involving 0 elements.


Best and SoTA practice is to flatten the image locally. I.e. we flatten only the receptive field of the kernel eg. 3x3x3, and then matmul it with our flattened kernel(3x3x3). This results in matmul between a 1x27 image and 27x1 kernel, resulting in a single scalar value. This scalar corresponds to a single pixel of the activation map after being added to the scalar bias term.

Let's take an example. Start from inputs of dimension 16x3x32x32 where 16 is the batch size, 3 is the number of channels and 32x32 is the image size. So we have 16 rgb images of 32x32 resolution as input each batch. Consider a kernel size of 128x3x2x2 where 128 is the number of kernels, 3 represents rgb and 3x3 is the kernel size. 

How does im2col work in this case? 

Firstly, calculate the receptive field size or the output field size.

O = ((I - K + 2P)/S) + 1

This is the standard formula where:
- I : Input image size (eg. 32x32)
- K : Kernel size (eg. 3x3)
- P : Zero Padding size (eg. 0 or 1 if we want O to be 32 = I)
- S : Stride (eg. 1)
- O : Output Activation size

Here, 

O = ((32-3+0)/1) + 1 = 30

So we get activation map of 30x30 per kernel.

If we want the activation map to be same size of that of input, we have to introduce padding: Specifically same padding. 

In our example, same padding requires padding = 1

Therefore we get, O = ((32-3+2)/1) + 1 = 32

For now, let us assume the initial case with no Padding. 

## Im2col Transformation Step-by-Step

### Given:
- Input: (16, 3, 32, 32) - 16 batches, 3 channels, 32×32 images
- Kernels: (128, 3, 3, 3) - 128 output channels/kernels/activation maps, 3 input channels, 3×3 spatial
- Output size: O = ((32 - 3 + 0) / 1) + 1 = **30×30**

### Step 1: Extract All Receptive Fields

For each image in the batch:
- Each receptive field: 3×3×3 = **27 elements**
- Number of positions: 30×30 = **900 positions**
- Total number of flattened matrices that represent all positions: **(900, 27)** (But this flatenning requires special handling so that we differentiate from 27 contigious memory of the image, and 27 neighbouring pixels that constitute a receptive field.)

For the entire batch (16 images):
- We stack all of them vertically (16 × 900, 27) = **(14,400, 27)**

### Step 2: Flatten All 128 Kernels

- Each kernel: 3×3×3 = 27 elements
- Kernels shape: **(27, 128)**

Why? 27 weights for each 128 kernels. 

### Step 3: Giant MatMul

(14,400, 27) @ (27, 128) = **(14,400, 128)**

### Step 4: Reshape and Add Bias

Reshape (14,400, 128) → (16, 128, 30, 30)
Add bias (128,) → broadcasts to (16, 128, 30, 30)


## So lets draw some conclusions first!

1. We need special type of matrix flatenning such that all pixels of local receptive fields(3x3x3 = 27) are in one dimension(x, 27) and all such local receptive fields(30x30 = 900) are in another, batch dimension(900, x) which represents the number of such possible receptive fields. This corresponds to number of pixels in activation map. Finally we need to consider all the receptive fields of all images in the batch. This doesnot create new dimension (not, eg. 16x900x27), but we stack all the receptive fields of all the images vertically. (i.e 900x16 = 14,400). Therefore we get final flatenned 2D matrix of shape = 14400, 27.

    ## Therefore:
    Local receptive field pixels must be arranged non contigiously relative to the image.

    eg.

    11  12  13

    21  22  23

    31  32  33

    should not be flattened as: 

    [11, 12, 13, 14],
        
    [13, 21, 22, 23], 


    [22, 23, 31, 32], 

    [31, 32, 33, 11] 

    Shape(4x4)

    or,

    [1,2,3,4,5,6,7,8,9] : Shape (9)

    But as:
    
    [11, 12, 21, 22],

    [12, 13, 22, 23],

    [21, 22, 23, 31],

    [22, 23, 32, 33]

    Shape(4x4)


2. To define our convolution layer, we need to know some parameters:
- Depth/No. of Channels of Kernel which corresponds to input channels(in_channels)
- Depth/No. of Channels of Activation/Output which corresponds to number of kernels (out_channels)
- Kernel size (kernel_size eg.(2) -> 2x2) 

In [10]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        #assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        #self.rows = fromArray.shape[0]
        #self.columns = fromArray.shape[1]
        self.shape = fromArray.shape
        self._prev = set(_children)
        self._operation = _operation
        self._backward = lambda : None
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, shape, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros(shape, dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    @classmethod
    def random(cls, shape, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.randn(*shape) * 0.1).astype(dtype=dtype)
        t.shape = shape
        return t
    
    @classmethod
    def const(cls, shape, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full(shape, constant)).astype(dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    #Operations
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            out1 = self.return_unbroadcasted(out)
            out2 = other.return_unbroadcasted(out)
            self.grad += out1 #Derivation in the notes. 
            other.grad += out2
        out = tensor(out_matrix, (self, other), '+')
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        return self + (-1 * other)
    
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        return other + (-1 * other)
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            out1 = self.return_unbroadcasted(out)
            out2 = other.return_unbroadcasted(out)
            self.grad += out1* other.matrix #Derivation in the notes. 
            other.grad += out2 * self.matrix

        out = tensor(out_matrix, (self, other), '*')
        out._backward = _backward
        return out
    
    '''
    batch multiplication might cause shape broadcasts.
    eg. (3,2,2) @ (1,2,3) = (3,2,3)
    this is similar to our element wise operations
    thus we should be handling this the same way we did for elementwise operations
    But, for now, we would be working in a controlled way (Even for CNNS)
    and wouldn't need this handling.
    '''
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape()[0] == self.shape()[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            self.grad += out.grad @ (other.matrix).swapaxes(-2,-1)#Derivation in the notes.
            other.grad += (self.matrix).swapaxes(-2,-1) @ out.grad 
        out = tensor(out_matrix, (self, other), '@')
        out._backward = _backward
        return out
    

    #I and thus we should learn at this point that to make our class compatible for ND tensors,
    #We need the matrix multiplication and Transpose backward to change
    #For higher dimensions, matmul = batch matmul where multiplication is done 
    #along each and every batches of 2D matrix. 
    #eg. If we have (2,3,3) shape tensor, it implies there are two batches of (3,3) matrices
    #similarly, (2,3,3,2) shape = 2x3 batches of 3x2 matrices.
    #matrix multiplication, (2,3,3) @ (2,3,2) = (2,3,2)
    def swap_axes(self, axis1, axis2):
        out_matrix = self.matrix.swapaxes(axis1, axis2)
        
        def _backward():
            self.grad = np.zeros_like(out.grad.swapaxes(axis1,axis2)) if self.grad is None else self.grad
            self.grad += (out.grad).swapaxes(axis1,axis2) #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out

    def transpose(self):
        out_matrix = self.matrix.transpose()
        
        def _backward():
            self.grad = np.zeros_like(out.grad.transpose()) if self.grad is None else self.grad
            self.grad += (out.grad).transpose() #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out
    
    def __rmatmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        return other @ self
    
    def __pow__(self, N):
        assert isinstance(N, int | float), "Can only power up by scalars!"
        out_matrix = self.matrix ** N

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            out1 = self.return_unbroadcasted(out)
            self.grad += N * (self.matrix ** (N-1)) * out1
        
        out = tensor(out_matrix, _children=(self, ), _operation="**")
        out._backward = _backward
        return out
    
    def __truediv__(self, other):
        other = self.checkOther(other)
        return self * (other**-1)
    
    def __rtruediv__(self, other):
        return other * (self**-1)
    
    def sum(self):
        out_matrix = np.array(([[self.matrix.sum()]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad

        out = tensor(out_matrix, _children=(self, ), _operation='sum()')
        out._backward = _backward
        return out

    def mean(self):
        N = np.prod(self.shape)
        out_matrix = np.array(([[self.matrix.sum()/(N)]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad / N

        out = tensor(out_matrix, _children=(self, ), _operation='mean()')
        out._backward = _backward
        return out
    
    def ReLU(self):
        out_matrix = np.maximum(0,self.matrix)

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += (self.matrix > 0).astype(self.matrix.dtype) * out.grad

        out = tensor(out_matrix, (self, ), "ReLU")
        out._backward = _backward
        return out
    
    def reshape(self, shape):
        assert isinstance(shape, tuple), f"Can only reshape using shape tuples e.g. (3,3). Provided is {shape}"
        out_matrix = self.matrix.reshape(shape)

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += out.grad.reshape(self.shape)

        out = tensor(out_matrix, (self, ), "reshape()")
        out._backward = _backward
        return out
    
    def flatten(self):
        out_matrix = self.matrix.reshape(-1,np.prod(self.shape[1:]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += out.grad.reshape(self.shape)

        out = tensor(out_matrix, (self, ), "flatten()")
        out._backward = _backward
        return out
    
    #Helper Functions
    #def shape(self):
     #   return (self.rows, self.columns)

    def return_unbroadcasted(self, out):  
        added_axis = []
        stretched_axis = []
        for index, (first_no, second_no) in enumerate(itertools.zip_longest(reversed(self.shape), reversed(out.shape))):
            if first_no is None:
                added_axis.append(index)
            elif (first_no == 1) and (second_no > 1):
                stretched_axis.append(index)
        grad = out.grad
        ndim = len(out.shape)
        if stretched_axis:
            original_axes = tuple(ndim - 1 - i for i in stretched_axis)
            grad = np.sum(grad, axis=original_axes, keepdims=True)
        if added_axis:
            original_axes = tuple(ndim - 1 - i for i in added_axis)
            grad = np.sum(grad, axis=original_axes, keepdims=False)
        return grad

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.shape, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        #assert other.shape == self.shape, "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        
    def backward(self):
        self.grad = np.ones_like(self.matrix, dtype=float)
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        for current in reversed(topo):

            current._backward()

    __array_ufunc__ = None

In [66]:
class Conv2d:
    def __init__(self, in_channels, out_channels, kernel_size):
        self.kernel = tensor.random((out_channels, in_channels, kernel_size, kernel_size))

    @classmethod
    def im2col(cls, X : tensor, kernel_size, stride):

        batch_size = X.shape[0]
        channels = X.shape[1]
        image_height = X.shape[-2] #Rows
        image_width = X.shape[-1] #Columns


        #We are assuming square kernels.
        kernel_h = kernel_size
        kernel_w = kernel_size

        act_h = (((image_height - kernel_size)//stride) + 1) #height of activation
        act_w = (((image_width - kernel_size)//stride) + 1)  #width of activation

        istrides = X.matrix.strides #strides of input tensor

        intermediate_6D = np.lib.stride_tricks.as_strided(
                            X.matrix,
                            shape=(batch_size, act_h, act_w, channels, kernel_h, kernel_w),
                            strides=(istrides[0], #No of images stride bytes
                                     istrides[-2] * stride, #Activation map Vertical stride bytes
                                     istrides[-1] * stride, #Activation map Horizontal stride bytes
                                     istrides[1], #Channel stride bytes
                                     istrides[-2], #Rective field vertical stride bytes
                                     istrides[-1]) #Receptive field horizontal stride bytes
                            )
        
        out_shape = (batch_size * act_h * act_w, channels * kernel_h * kernel_w)
        out_matrix = np.reshape(intermediate_6D, shape=out_shape)


        def _backward():
            X.grad = np.zeros_like(X.matrix) if X.grad is None else X.grad
            grad_6D = np.reshape(out.grad, shape=(batch_size, act_h, act_w, channels, kernel_h, kernel_w,))

            #For each element in this 6D tensor, having 6D index, we have to calculate the coresponding 4D index.
            #The formula has been conceptually derived in the notes.
            #Here, we first generate all the indices of the 6D tensor and store each index dimension in separate list
            #Then using the derived formula, we batch convert the 6D indices to 4D indices.

            batch = np.arange(batch_size).reshape(batch_size,1,1,1,1,1)
            field_h = np.arange(act_h).reshape(1,act_h,1,1,1,1)
            field_w = np.arange(act_w).reshape(1,1,act_w,1,1,1)
            channel = np.arange(channels).reshape(1,1,1,channels,1,1)
            k_h = np.arange(kernel_h).reshape(1,1,1,1,kernel_h,1)
            k_w = np.arange(kernel_w).reshape(1,1,1,1,1,kernel_w)

            x = stride * field_h + k_h
            y = stride * field_w + k_w

            np.add.at(X.grad, (batch, channel, x, y), grad_6D)

        out = tensor(out_matrix, _children=(X, ), _operation='im2col')
        out._backward = _backward

        return out
        

In [36]:
batch = tensor(np.random.randn(16, 3, 5, 5).astype(float))

print("Input Shape: ", batch.shape)

im2col = Conv2d.im2col(batch, 3, 2)

print("Output Shape: ", im2col.shape)

Input Shape:  (16, 3, 5, 5)
Output Shape:  (64, 27)


This was a valid approach. Elegant and shows the direct use of formula. But, we have an even more efficient approach. It involves slicing. In this method, instead of creating large index array which requires both time and memory, we can call 2 simple for loops to loop over the kernel sizes and slice the out.grad array for each position of the kernel index.

In [58]:
class Conv2d:
    def __init__(self, in_channels, out_channels, kernel_size):
        self.kernel = tensor.random((out_channels, in_channels, kernel_size, kernel_size))

    @classmethod
    def im2col(cls, X : tensor, kernel_size, stride):

        batch_size = X.shape[0]
        channels = X.shape[1]
        image_height = X.shape[-2] #Rows
        image_width = X.shape[-1] #Columns


        #We are assuming square kernels.
        kernel_h = kernel_size
        kernel_w = kernel_size

        act_h = (((image_height - kernel_size)//stride) + 1) #height of activation
        act_w = (((image_width - kernel_size)//stride) + 1)  #width of activation

        istrides = X.matrix.strides #strides of input tensor

        intermediate_6D = np.lib.stride_tricks.as_strided(
                            X.matrix,
                            shape=(batch_size, act_h, act_w, channels, kernel_h, kernel_w),
                            strides=(istrides[0], #No of images stride bytes
                                     istrides[-2] * stride, #Activation map Vertical stride bytes
                                     istrides[-1] * stride, #Activation map Horizontal stride bytes
                                     istrides[1], #Channel stride bytes
                                     istrides[-2], #Rective field vertical stride bytes
                                     istrides[-1]) #Receptive field horizontal stride bytes
                            )
        
        out_shape = (batch_size * act_h * act_w, channels * kernel_h * kernel_w)
        out_matrix = np.reshape(intermediate_6D, shape=out_shape)


        def _backward():
            X.grad = np.zeros_like(X.matrix) if X.grad is None else X.grad
            
            grad_6D = out.grad.reshape(batch_size, act_h, act_w, channels, kernel_h, kernel_w)

            for i in range(kernel_h):
                for j in range(kernel_w):
                    # 1. Extract the gradient slice for this kernel position
                    grad_slice = grad_6D[:, :, :, :, i, j]
                    
                    grad_slice_transposed = grad_slice.transpose(0, 3, 1, 2)
                    
                    X.grad[:, :, 
                        i : i + act_h * stride : stride, 
                        j : j + act_w * stride : stride
                    ] += grad_slice_transposed

        out = tensor(out_matrix, _children=(X, ), _operation='im2col')
        
        out._backward = _backward

        return out
        

## Testing im2col Forward and Backward

Let's verify that both the forward and backward passes work correctly with a concrete example.

In [64]:
# Test 1: Forward pass with simple input
print("=" * 70)
print("TEST 1: Forward Pass")
print("=" * 70)

# Create a simple input: (2, 1, 5, 5) - 2 batches, 1 channel, 5x5 image
X_test = tensor(np.arange(50).reshape(2, 1, 5, 5).astype(float))
print(f"Input shape: {X_test.shape}")
print(f"Input (batch 0):\n{X_test.matrix[0, 0]}")
print()

# Apply im2col with kernel_size=2, stride=2
out_test = Conv2d.im2col(X_test, kernel_size=2, stride=2)
print(f"Output shape: {out_test.shape}")
print(f"Expected: (2 * 2 * 2, 1 * 2 * 2) = (8, 4)")
print()

# Check the output values
print(f"Output matrix (first few rows):")
print(out_test.matrix[:4])
print()

# Manually verify the first patch (top-left of batch 0)
print("Manual verification - First patch (batch 0, position [0,0]):")
print("Should contain pixels at positions (0,0), (0,1), (1,0), (1,1)")
first_batch = X_test.matrix[0, 0]
print(f"Expected: [{first_batch[0,0]}, {first_batch[0,1]}, {first_batch[1,0]}, {first_batch[1,1]}]")
print(f"Got:      {out_test.matrix[0]}")
print()

TEST 1: Forward Pass
Input shape: (2, 1, 5, 5)
Input (batch 0):
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]

Output shape: (8, 4)
Expected: (2 * 2 * 2, 1 * 2 * 2) = (8, 4)

Output matrix (first few rows):
[[ 0.  1.  5.  6.]
 [ 2.  3.  7.  8.]
 [10. 11. 15. 16.]
 [12. 13. 17. 18.]]

Manual verification - First patch (batch 0, position [0,0]):
Should contain pixels at positions (0,0), (0,1), (1,0), (1,1)
Expected: [0.0, 1.0, 5.0, 6.0]
Got:      [0. 1. 5. 6.]



In [67]:
# Test 2: Backward pass - verify gradients flow correctly
print("=" * 70)
print("TEST 2: Backward Pass")
print("=" * 70)

# Create a simple input
X_back = tensor(np.ones((1, 1, 4, 4), dtype=float))
print(f"Input shape: {X_back.shape}")
print(f"Input values (all ones):\n{X_back.matrix[0, 0]}")
print()

# Forward: kernel_size=2, stride=1 → output is 3x3
out_back = Conv2d.im2col(X_back, kernel_size=2, stride=1)
print(f"Output shape: {out_back.shape}")
print(f"Expected: (1 * 3 * 3, 1 * 2 * 2) = (9, 4)")
print()

# Create gradient: all ones for simplicity
out_back.grad = np.ones_like(out_back.matrix)
print(f"Output gradient (all ones): shape {out_back.grad.shape}")
print()

# Call backward
out_back._backward()

print(f"Input gradient shape: {X_back.grad.shape}")
print(f"Input gradient:\n{X_back.grad[0, 0]}")
print()

# Explanation: With kernel 2x2 and stride 1, each position in the input
# contributes to multiple output positions (overlapping receptive fields)
print("Expected gradient pattern:")
print("  Corner pixels (e.g., [0,0]): contribute to 1 output position → grad = 1")
print("  Edge pixels: contribute to 2 output positions → grad = 2")  
print("  Interior pixels: contribute to 4 output positions → grad = 4")
print()

# Verify corner [0,0]
print(f"Gradient at corner [0,0]: {X_back.grad[0, 0, 0, 0]} (expected: 1)")
# Verify edge [0,1] or [1,0]  
print(f"Gradient at edge [0,1]: {X_back.grad[0, 0, 0, 1]} (expected: 2)")
# Verify interior [1,1]
print(f"Gradient at interior [1,1]: {X_back.grad[0, 0, 1, 1]} (expected: 4)")

TEST 2: Backward Pass
Input shape: (1, 1, 4, 4)
Input values (all ones):
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]

Output shape: (9, 4)
Expected: (1 * 3 * 3, 1 * 2 * 2) = (9, 4)

Output gradient (all ones): shape (9, 4)

Input gradient shape: (1, 1, 4, 4)
Input gradient:
[[1. 2. 2. 1.]
 [2. 4. 4. 2.]
 [2. 4. 4. 2.]
 [1. 2. 2. 1.]]

Expected gradient pattern:
  Corner pixels (e.g., [0,0]): contribute to 1 output position → grad = 1
  Edge pixels: contribute to 2 output positions → grad = 2
  Interior pixels: contribute to 4 output positions → grad = 4

Gradient at corner [0,0]: 1.0 (expected: 1)
Gradient at edge [0,1]: 2.0 (expected: 2)
Gradient at interior [1,1]: 4.0 (expected: 4)


In [62]:
# Test 3: Verify slicing approach with stride > 1
print("=" * 70)
print("TEST 3: Stride > 1 Case")
print("=" * 70)

# Input: (1, 1, 5, 5)
X_stride = tensor(np.arange(25).reshape(1, 1, 5, 5).astype(float))
print(f"Input shape: {X_stride.shape}")
print(f"Input:\n{X_stride.matrix[0, 0]}")
print()

# Forward: kernel_size=2, stride=2 → output is 2x2
out_stride = Conv2d.im2col(X_stride, kernel_size=2, stride=2)
print(f"Output shape: {out_stride.shape}")
print(f"Expected: (1 * 2 * 2, 1 * 2 * 2) = (4, 4)")
print()
print(f"Output patches:")
print(out_stride.matrix)
print()

# Each row should correspond to one 2x2 patch
print("Verification:")
print(f"Patch at [0,0]: positions (0,0), (0,1), (1,0), (1,1) → {X_stride.matrix[0,0,0:2,0:2].flatten()}")
print(f"Got: {out_stride.matrix[0]}")
print()
print(f"Patch at [0,2]: positions (0,2), (0,3), (1,2), (1,3) → {X_stride.matrix[0,0,0:2,2:4].flatten()}")
print(f"Got: {out_stride.matrix[1]}")
print()

# Test backward with stride=2
out_stride.grad = np.ones_like(out_stride.matrix)
out_stride._backward()

print(f"Input gradient with stride=2:")
print(X_stride.grad[0, 0])
print()
print("Expected: Each receptive field position gets gradient 1")
print("With stride=2, no overlaps → all touched positions should have grad=1")
print("Untouched positions (e.g., last row/col) should have grad=0")

TEST 3: Stride > 1 Case
Input shape: (1, 1, 5, 5)
Input:
[[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19.]
 [20. 21. 22. 23. 24.]]

Output shape: (4, 4)
Expected: (1 * 2 * 2, 1 * 2 * 2) = (4, 4)

Output patches:
[[ 0.  1.  5.  6.]
 [ 2.  3.  7.  8.]
 [10. 11. 15. 16.]
 [12. 13. 17. 18.]]

Verification:
Patch at [0,0]: positions (0,0), (0,1), (1,0), (1,1) → [0. 1. 5. 6.]
Got: [0. 1. 5. 6.]

Patch at [0,2]: positions (0,2), (0,3), (1,2), (1,3) → [2. 3. 7. 8.]
Got: [2. 3. 7. 8.]

Input gradient with stride=2:
[[1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0.]]

Expected: Each receptive field position gets gradient 1
With stride=2, no overlaps → all touched positions should have grad=1
Untouched positions (e.g., last row/col) should have grad=0


## Summary: im2col Forward & Backward Work Correctly! ✓

**Forward Pass:**
- Correctly extracts all receptive fields using `as_strided`
- Reshapes to 2D for efficient matrix multiplication
- Example: (1, 1, 5, 5) with kernel=2, stride=2 → (4, 4) ✓

**Backward Pass (Slicing Approach):**
- Correctly accumulates gradients back to input
- Handles overlapping receptive fields (stride=1): interior pixels get 4× gradient ✓
- Handles non-overlapping receptive fields (stride=2): touched pixels get 1×, untouched get 0× ✓
- Uses efficient slicing: `X.grad[:, :, i:i+act_h*stride:stride, j:j+act_w*stride:stride] += grad_slice_transposed`

**Key Insight:**
The slicing approach works because each `(i, j)` kernel position maps to a specific strided slice of the input. The slice shape `(batch, channels, act_h, act_w)` matches `grad_slice_transposed`, so the `+=` operation is valid. Across all kernel positions, these slices cover the entire input (with overlaps when stride < kernel_size).